# Optimizing scooter pickup

In this interactive example, we will see how to use the rideOS `Fleet Planner API` to solve a fleet optimization problem. We also compare Fleet Planner's solution to that from a greedy optimizer.

## The Scenario

Let's say that we operate an electric scooter sharing service, and we have a number of scooters scattered around San Francisco that need to be moved to a depot as quickly as possible so they can be recharged. We have a smaller number of couriers who are able to drive around the city and collect the scooters. Each courier can carry a limited number of scooters, so they'll have to make multiple trips to the depot. For each courier, we want to determine a plan - i.e. an ordered list of waypoints, where each waypoint corresponds to picking up a scooter or dropping it off at the depot - such that all scooters are moved to the depot as quickly as possible.

Note that the the purposes of this exercise, we'll assume that the couriers are able to pickup/dropoff a scooter instantly once they arrive at the pickup/dropoff location.

## Dependencies

For this example we will use `NumPy`, `requests` and `matplotlib` together with `mplleaflet` to display the results on an interactive map.

In [1]:
!pip install -q mplleaflet
!pip install -q haversine

import numpy as np
import matplotlib.pyplot as plt
import mplleaflet
from haversine import haversine
import requests
import os

## API Key

To run this example, you'll need a rideOS API key. You can sign up for one [here](https://app.rideos.ai/) and view it on your [profile page](https://app.rideos.ai/profile). Then, assign it to the `API_KEY` variable below:

In [2]:
# IMPORTANT: replace "YOUR_RIDEOS_API_KEY" with your actual rideOS API key
API_KEY = "YOUR_RIDEOS_API_KEY"

AUTHORIZATION_HEADER = {"X-Api-Key": API_KEY}

## Initial state

First, we load the initial scooter locations from a CSV file. The scooter locations were randomly generated from the uniform distribution. We manually define the location of the depot and couriers. Note that we represent locations as WGS84 (latitude, longitude) tuples.

In [3]:
# The locations of the scooters that need to be picked up by the couriers and taken to the depot 
# for charging. List of (latitude, longitude) tuples
!wget -q https://raw.githubusercontent.com/rideOS/examples/master/scooters/scooter_locations.csv # Download the CSV from GitHub
SCOOTER_LOCATIONS = [(lat, lon) for lat, lon in np.loadtxt(open("scooter_locations.csv", "r"), delimiter=",", skiprows=1)]

# The location (latitude, longitude) where the scooters need to be dropped off for charging
DEPOT_LOCATION = (37.754120, -122.385931)

# The couriers available to pickup scooters and take them to the dropoff location
COURIERS = {
    "Mission Courier": {
        "startLocation": (37.752404, -122.418420),
        "scooterCapacity": 5,
        "color": "blue"
    },
    "Richmond Courier": {
        "startLocation": (37.780156, -122.480945),
        "scooterCapacity": 5,
        "color": "green"
    }
}

# Plots the initial state (courier start locations, scooter locations, depot location) on a map
def plot_initial_state():
    for courier in COURIERS.values():
            plt.plot(courier["startLocation"][1], courier["startLocation"][0], color=courier["color"], marker="s")
    plt.plot([lon for lat, lon in SCOOTER_LOCATIONS], [lat for lat, lon in SCOOTER_LOCATIONS], "rv")
    plt.plot(DEPOT_LOCATION[1], DEPOT_LOCATION[0], "g^")

plot_initial_state()
mplleaflet.display(tiles='cartodb_positron')

## Convert the initial state into a request to the Fleet Planner API

Now that we've defined our initial state, we generate a Fleet Planner request that we'll run in the next cell. Each courier is a "vehicle" in the request, and each scooter is a "task" (pick up the scooter at its location and drop it off at the depot location).

In [4]:
import json

# Generates a Fleet Planner vehicle for the specified courier
def generate_vehicle(courier):
    return {
        "position": lat_lon_tuple_to_position_dict(courier["startLocation"]),
        "resourceCapacity": courier["scooterCapacity"]
    }

# Generates a Fleet Planner task with the specified pickup and dropoff locations
def generate_task(pickup_location, dropoff_location):
    return {
        "resourcesRequired": 1,
        "pickupStep": {
            "position": lat_lon_tuple_to_position_dict(pickup_location)
        },
        "dropoffStep": {
            "position": lat_lon_tuple_to_position_dict(dropoff_location)
        }
    }

# Converts a tuple in the form (latitude, longitude) to a dict of the form that will be used in rideOS's API requests
def lat_lon_tuple_to_position_dict(lat_lon_tuple):
    return {
        "latitude": lat_lon_tuple[0],
        "longitude": lat_lon_tuple[1]
    }

# Compute the "vehicles" field for our Fleet Planner request. Each courier is a vehicle
fleet_planner_request_vehicles = dict((courier_name, generate_vehicle(courier)) for courier_name, courier in COURIERS.items())

# Compute the "tasks" field for our Fleet Planner request. Each scooter generates one task (pickup the scooter
# at its location and dropoff at the depot location)
fleet_planner_request_tasks = dict(("Scooter %02d pickup/dropoff" % (i), generate_task(pickup_location, DEPOT_LOCATION)) for i, pickup_location in enumerate(SCOOTER_LOCATIONS))

fleet_planner_request = {
    "vehicles": fleet_planner_request_vehicles,
    "tasks": fleet_planner_request_tasks
}

print("Generated the following Fleet Planner request:")
print(json.dumps(fleet_planner_request, indent=4))


Generated the following Fleet Planner request:
{
    "vehicles": {
        "Mission Courier": {
            "position": {
                "latitude": 37.752404,
                "longitude": -122.41842
            },
            "resourceCapacity": 5
        },
        "Richmond Courier": {
            "position": {
                "latitude": 37.780156,
                "longitude": -122.480945
            },
            "resourceCapacity": 5
        }
    },
    "tasks": {
        "Scooter 00 pickup/dropoff": {
            "resourcesRequired": 1,
            "pickupStep": {
                "position": {
                    "latitude": 37.76910042512045,
                    "longitude": -122.42480099392432
                }
            },
            "dropoffStep": {
                "position": {
                    "latitude": 37.75412,
                    "longitude": -122.385931
                }
            }
        },
        "Scooter 01 pickup/dropoff": {
            "resourcesRe

## Call Fleet Planner for a solution

Now, we pass the request we generated to Fleet Planner. Note that because Fleet Planner factors in real-time data such as traffic conditions, you won't always get exactly the same response.

In [5]:
GET_FLEET_PLAN_URL = "https://api.rideos.ai/fleet/v2/GetPlan"

print("Calling fleet planner to generate an optimal plan for the couriers...")
response = requests.post(
    GET_FLEET_PLAN_URL,
    headers=AUTHORIZATION_HEADER,
    json=fleet_planner_request
)
response.raise_for_status()

fleet_planner_response = response.json()

print("Got successful fleet planner response:")
print(json.dumps(fleet_planner_response, indent=4))

Calling fleet planner to generate an optimal plan for the couriers...
Got successful fleet planner response:
{
    "recommendations": [
        {
            "vehicleId": "Mission Courier",
            "planRecommendation": {
                "assignedSteps": [
                    {
                        "taskId": "Scooter 03 pickup/dropoff",
                        "stepType": "PICKUP"
                    },
                    {
                        "taskId": "Scooter 01 pickup/dropoff",
                        "stepType": "PICKUP"
                    },
                    {
                        "taskId": "Scooter 06 pickup/dropoff",
                        "stepType": "PICKUP"
                    },
                    {
                        "taskId": "Scooter 04 pickup/dropoff",
                        "stepType": "PICKUP"
                    },
                    {
                        "taskId": "Scooter 00 pickup/dropoff",
                        "stepType": "PICKU

## Display Fleet Planner solution

Displays the Fleet Planner solution on a map, including the detailed path that each courier will take to perform their assigned tasks.

In [6]:
GET_PATH_URL = "https://api.rideos.ai/path/v2/GetPath"

# Summarizes and displays a solution to the optimization problem. The solution should be provided as a dict that
# maps from courier name to a list of waypoints that the courier should traverse. The first waypoint is assumed
# to be the courier's start location, and each subsequent waypoint will either be a scooter location (if the
# courier is picking up a scooter at that waypoint) or the depot location (if the courier is dropping off scooters
# at that waypoint).
def summarize_and_display_solution(courier_name_to_waypoints_dict):
    max_time = 0
    for courier_name, waypoints in courier_name_to_waypoints_dict.items():
        # Fetch the optimal path through the steps that the vehicle will perform
        path = fetch_path_through_waypoints(waypoints)["paths"][0]
        
        # Because we specified "type": "PASS_THROUGH" on our Path request in fetch_path_through_waypoints(),
        # the response will contain a single leg that passes through all of the waypoints
        linestring = path["legs"][0]["lineString"]["positions"]

        max_time = max(get_duration_in_seconds(path["travelTime"]), max_time)

        plt.plot(
            [location["longitude"] for location in linestring],
            [location["latitude"] for location in linestring],
            color=COURIERS[courier_name]["color"]
        )

    print("Time required to pickup and dropoff all scooters: %f minutes" % (max_time / 60))

# Calls rideOS's Path API to retrieve the optimal path through the specified waypoints. This takes into account
# real-time data such as traffic conditions
def fetch_path_through_waypoints(waypoint_locations):
    response = requests.post(
        GET_PATH_URL,
        headers=AUTHORIZATION_HEADER,
        json={
            "waypoints": [{ "position": lat_lon_tuple_to_position_dict(loc), "type": "PASS_THROUGH"} for loc in waypoint_locations],
            "geometryFormat": "LINESTRING"
        }
    )
    response.raise_for_status()
    return response.json()

# Convert a duration string such as "3.14159s" into the floating point duration in seconds
def get_duration_in_seconds(duration):
    # durations will always be represented as strings that represent the number of seconds and end in "s".
    # ex: "3.14159s"
    return float(duration.split("s")[0])

def get_assigned_step_position(assigned_step, fleet_planner_request_tasks):
    position = None
    if assigned_step["stepType"] == "PICKUP":
        position = fleet_planner_request_tasks[assigned_step["taskId"]]["pickupStep"]["position"]
    else:
         position = fleet_planner_request_tasks[assigned_step["taskId"]]["dropoffStep"]["position"]
    return (position["latitude"], position["longitude"])

# Generate a dict (courier name -> waypoints for that courier) that we can pass to summarize_and_display_solution() to display
fleet_planner_courier_name_to_waypoints_dict = dict(
    (
        recommendation["vehicleId"], 
        [COURIERS[recommendation["vehicleId"]]["startLocation"]] + [get_assigned_step_position(assigned_step, fleet_planner_request_tasks) for assigned_step in recommendation["planRecommendation"]["assignedSteps"]]
    )
    for recommendation in fleet_planner_response["recommendations"]
)

print("Summarizing and plotting the solution...")
summarize_and_display_solution(fleet_planner_courier_name_to_waypoints_dict)
plot_initial_state()
mplleaflet.display(tiles='cartodb_positron')

Summarizing and plotting the solution...
Time required to pickup and dropoff all scooters: 227.139633 minutes


## Generate and display greedy solution for comparison

Here we generate a solution to the scooter pickup problem using a simple, greedy algorithm that works as follows: on each iteration, we compute the closest possible task to each courier. Of those possible tasks, we pick the one that's closest to its courier, and have that courier "execute" the task. We repeat until all tasks have been completed (i.e. all scooters have been picked up and dropped off at the depot).

In [7]:
from haversine import haversine

# Represents the state of a courier as the greedy solver progresses
class CourierState:
    def __init__(self, name, start_location, scooter_capacity, depot_location):
        self._name = name
        self._waypoints = [start_location]
        self._scooter_capacity = scooter_capacity
        self._scooter_count = 0
        self._depot_location = depot_location

    # Finds the closest task to the courier
    def get_closest_task(self, scooter_locations, scooter_states):
        if False in scooter_states:
            # There are scooters that still need to be picked up, so greedily try to pick them up until we reach our
            # carrying capacity, at which point head to the depot to drop them all off
            if self._scooter_count < self._scooter_capacity:
                closest_scooter_index = self.__find_closest_available_scooter_index(scooter_locations, scooter_states)
                return {
                    "type": "PICKUP", 
                    "start": self._waypoints[-1],
                    "end": scooter_locations[closest_scooter_index],
                    "scooter_index": closest_scooter_index
                }
            else:
                return {
                    "type": "DROPOFF",
                    "start": self._waypoints[-1],
                    "end": self._depot_location
                }
        else:
            # all scooters have been picked up, so the only thing left to do is dropoff any scooters that we're
            # currently carrying
            if self._scooter_count > 0:
                return {
                    "type": "DROPOFF",
                    "start": self._waypoints[-1],
                    "end": self._depot_location
                }
            else:
                # Nothing left to do for this courier
                return None

    def execute_task(self, task):
        self._waypoints.append(task["end"])
        if task["type"] == "PICKUP":
            # We've picked up a new scooter
            self._scooter_count += 1
        else:
            # We've dropped off all of our scooters
            self._scooter_count = 0

    def __find_closest_available_scooter_index(self, scooter_locations, scooter_states):
        available_scooter_indices = [i for i, p in enumerate(scooter_states) if p == False]        
        available_scooter_distances = [haversine(self._waypoints[-1], scooter_locations[i]) for i in available_scooter_indices]
        min_i = np.argmin(available_scooter_distances)
        return available_scooter_indices[min_i]
    
def get_closest_possible_task(courier_states, scooter_states):
    # Fetch the closest possible task for each courier
    couriers_and_tasks = [(courier_state, courier_state.get_closest_task(SCOOTER_LOCATIONS, scooter_states)) for courier_state in courier_states]
    couriers_and_tasks = [(courier_state, task) for courier_state, task in couriers_and_tasks if task is not None]
    
    # We're done. All scooters have been picked up and dropped off at the depot
    if len(couriers_and_tasks) == 0:
        return (None, None)
    
    # Fetch the distance of each task to its courier
    distances = [haversine(task["start"], task["end"]) for courier_state, task in couriers_and_tasks if task is not None]
    
    min_i = np.argmin(distances)
    return couriers_and_tasks[min_i]

# The state of each scotter: just a boolean indicating whether the scooter has been picked up or not
scooter_states = [False] * len(SCOOTER_LOCATIONS)

# The state of each courier as the simulation progresses
courier_states = [CourierState(courier_name, courier["startLocation"], courier["scooterCapacity"], DEPOT_LOCATION) for courier_name, courier in COURIERS.items()]

print('Running greedy optimizer...')

# The basic algorithm works as follows: on each iteration, we compute the closest possible task to each 
# courier. Of those possible tasks, we pick the one that's closest to its courier, and have that courier
# "execute" the task. We repeat until all tasks have been completed (i.e. all scooters have been picked up
# and dropped off at the depot).
courier_state, closest_possible_task = get_closest_possible_task(courier_states, scooter_states)
while closest_possible_task is not None:
    # "execute" the task
    courier_state.execute_task(closest_possible_task)
    if closest_possible_task["type"] == "PICKUP":
        # Mark relevant scooter as having been picked up
        scooter_states[closest_possible_task["scooter_index"]] = True

    courier_state, closest_possible_task = get_closest_possible_task(courier_states, scooter_states)

print('Greedy optimizer finished. Displaying results...')

summarize_and_display_solution(dict((courier_state._name, courier_state._waypoints) for courier_state in courier_states))

plot_initial_state()
mplleaflet.display(tiles='cartodb_positron')

Running greedy optimizer...
Greedy optimizer finished. Displaying results...
Time required to pickup and dropoff all scooters: 350.700783 minutes


## Conclusion

As you can see from the results above, optimizing your fleet using rideOS's Fleet Planner API can be accomplished in a few lines of Python. In our sample run, Fleet Planner provided a solution that requires 35% less time than a greedy solution to move all of the scooters to the depot location for recharging (227 minutes vs. 350 minutes). If we were running a scooter sharing service, this would mean more scooters available more often for our customers.

This example barely scratched the surface of what Fleet Planner and rideOS's other APIs can provide. We hope that you'll take a look at our [API docs](https://app.rideos.ai/docs) for more info, and please [contact us](mailto:contact@rideos.ai) if you have any questions.